In [ ]:
%pip install redis
import redis
r = redis.Redis()
r.ping()

QST 7 et 8

In [6]:
import time

# Connect to the Redis server
r = redis.Redis(host='127.0.0.1', port=6379, db=0)

def check_user_permission(username, service):
    """
    Check if the user is authorized to access a specific service
    """
    # Check if the user is in the set of authorized users for the service
    if r.sismember(service, username):
        return True
    else:
        return False

def handle_login(username, password):
    """
    Handle a user login request
    """
    # Check if the user is in the set of authorized users for the service
    if r.sismember('authorized_users', username):
        # Check if the provided password matches the one stored in Redis
        if r.hget('users', username) == password:
            now = int(time.time())
            # Check if the user has exceeded the maximum number of connections
            # within the last 10 minutes
            if r.get(username + ":last_connection_time") and (now - int(r.get(username + ":last_connection_time")) < 600):
                if int(r.get(username + ":connection_count")) < 10:
                    r.incr(username + ":connection_count")
                    r.set(username + ":last_connection_time", now)
                    return True
                else:
                    return False
            else:
                r.set(username + ":connection_count", 1)
                r.set(username + ":last_connection_time", now)
                return True
        else:
            return False
    else:
        return False
    
def add_user_permission(username, service):
    """
    Add a user to the set of authorized users for a service
    """
    r.sadd(service, username)

def remove_user_permission(username, service):
    """
    Remove a user from the set of authorized users for a service
    """
    r.srem(service, username)


# Example usage

# Add a user to the set of authorized users for the Sale service
add_user_permission('user1', 'Sale')

# Check if the user is authorized to access the Sale service
print(check_user_permission('user1', 'Sale')) # Output: True

# Remove the user's permission to access the Sale service
remove_user_permission('user1', 'Sale')

# Check if the user is authorized to access the Sale service
print(check_user_permission('user1', 'Sale')) # Output: False


True
False


In [7]:
import redis

# Connect to the Redis server
r = redis.Redis(host='127.0.0.1', port=6379, db=1)

def log_service_call(username, service):
    """
    Log a service call made by a user
    """
    now = int(time.time())
    r.lpush(username + ":" + service, now)
    r.ltrim(username + ":" + service, 0, 9)

def get_user_service_calls(username, service):
    """
    Get the service calls made by a user
    """
    return r.lrange(username + ":" + service, 0, -1)

# Example usage

# Log a Sale service call made by user1
log_service_call('user1', 'Sale')

# Get the Sale service calls made by user1
print(get_user_service_calls('user1', 'Sale')) # Output: [1623472800]


[b'1674059778']
